In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

- Escoge 15 acciones del S&P 500 que esten listadas desde el 2015
- Crea 3 modelos, CAPM, 3 factores y 5 factores
- Obten coeficientes y R2
- Explica los resultados

In [2]:
start_date = datetime.datetime(1980, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

ff_factores = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)

factors_df = ff_factores[0] / 100
factors_df.index = factors_df.index.to_timestamp()

tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG',
           'JNJ', 'V', 'PG', 'JPM', 'XOM',
           'WMT', 'DIS', 'BAC', 'PFE', 'INTC']

stock_data = yf.download(tickers, start_date, end_date, interval='1mo')['Adj Close']
rt = stock_data.pct_change().dropna()

fama_french_df = pd.merge(rt, factors_df, on='Date')

def run_all_models(data, ticker):
    flag_data = data.copy()
    flag_data[ticker + '_minus_RF'] = data[ticker] - data['RF']
    y = flag_data[[ticker + '_minus_RF']]

    # CAPM
    X_capm = flag_data[['Mkt-RF']]
    model_capm = LinearRegression().fit(X_capm, y)
    prediction_capm = model_capm.predict(X_capm)
    r2_capm = r2_score(y_pred=prediction_capm, y_true=y)
    summary_capm = {str(X_capm.keys().values): list(model_capm.coef_[0])}

    # 3 factor model
    X_3factor = flag_data[['Mkt-RF', 'SMB', 'HML']]
    model_3factor = LinearRegression().fit(X_3factor, y)
    prediction_3factor = model_3factor.predict(X_3factor)
    r2_3factor = r2_score(y_pred=prediction_3factor, y_true=y)
    summary_3factor = {str(X_3factor.keys().values): list(model_3factor.coef_[0])}

    # 5 factor
    X_5factor = flag_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    model_5factor = LinearRegression().fit(X_5factor, y)
    prediction_5factor = model_5factor.predict(X_5factor)
    r2_5factor = r2_score(y_pred=prediction_5factor, y_true=y)
    summary_5factor = {str(X_5factor.keys().values): list(model_5factor.coef_[0])}

    summary = {
        'CAPM': {'BETAS': summary_capm, 'R2': r2_capm},
        '3 factor': {'BETAS': summary_3factor, 'R2': r2_3factor},
        '5 factor': {'BETAS': summary_5factor, 'R2': r2_5factor}

    }
    return summary

for i in tickers:
    print(i)
    display(run_all_models(fama_french_df, i))
    print()

<ipython-input-2-bec749f9066f>:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factores = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)
<ipython-input-2-bec749f9066f>:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factores = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)
[*********************100%%**********************]  15 of 15 completed


AAPL


{'CAPM': {'BETAS': {"['Mkt-RF']": [1.1143845634290752]},
  'R2': 0.4138521875472567},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [1.2571988031837507,
    -0.31806859551879263,
    -0.559538544976401]},
  'R2': 0.4875766452412814},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [1.258493086558058,
    -0.1592007338092657,
    -0.5561234698426883,
    0.555545962309884,
    -0.1066746927028863]},
  'R2': 0.5024098585968397}}


MSFT


{'CAPM': {'BETAS': {"['Mkt-RF']": [0.8928919055826889]},
  'R2': 0.4400744601027815},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [1.0571855488428104,
    -0.6587155977692553,
    -0.16705595654520072]},
  'R2': 0.5259423190103842},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [1.0101462856352508,
    -0.6863155657810771,
    0.037790135748025255,
    0.03375522888855714,
    -0.5169158466905799]},
  'R2': 0.5412191671356239}}

...

{'CAPM': {'BETAS': {"['Mkt-RF']": [1.788890040790411]},
  'R2': 0.4350342863536819},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [1.5785083553437718,
    0.0042622347446690245,
    1.5800387459376153]},
  'R2': 0.6049030634749466},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [1.3993492790628705,
    -0.6090444423420335,
    2.331146125042725,
    -1.6600364322221588,
    -1.5801997477486736]},
  'R2': 0.7016865011902563}}


PFE


{'CAPM': {'BETAS': {"['Mkt-RF']": [0.6358651511191276]},
  'R2': 0.24524964262552362},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [0.7204085487199821,
    -0.5079172137332644,
    0.18905470657254786]},
  'R2': 0.28678861028557545},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [0.7844650816868693,
    -0.3837850634165727,
    -0.08494334014358601,
    0.258643452650349,
    0.6377406520788806]},
  'R2': 0.32042802313709295}}


INTC


{'CAPM': {'BETAS': {"['Mkt-RF']": [0.9303899568747884]},
  'R2': 0.32869108203664676},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [0.9452320547562242,
    -0.09179378922767692,
    0.03746413516490371]},
  'R2': 0.32956134329534237},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [0.9557037556746885,
    -0.1456293050279709,
    -0.011571642831375392,
    -0.21861763765618605,
    0.1609373127136209]},
  'R2': 0.3329320366973313}}

En la gran mayoría de los casos se podría decir que, aunque el modelo de CAPM entrega un resultado medianamente bueno, es mejor el modelo de ajuste de 3 factores. Aunque representa un costo computacional más alto es considerable la mejora en el ajuste, por lo que el modelo de 3 factores es el ideal (en la mayoría de casos) por su relación de ajuste de error-costo computacional. El modelo de 5 factores presenta un sobre-ajuste ya que aunque si mejora el resultado del R2, no es significativo.